In [11]:
import pandas as pd
import os

report_path = os.getcwd() + "/reports/Monday-WorkingHours.pcap.report.csv"

In [12]:
df = pd.read_csv(report_path, index_col="id")

In [13]:
benign_count = df["Predictions"].value_counts().loc["BENIGN"]
mal_count = df["Predictions"].value_counts().loc["MALICIOUS"]

In [14]:
df["Predictions"].value_counts()

Predictions
BENIGN       324834
MALICIOUS     30898
Name: count, dtype: int64

In [15]:
df["Labels"] = 0

In [16]:
from sklearn.metrics import auc, precision_recall_curve, accuracy_score, precision_score, recall_score

def score_predictions(y_truth, y_predictions):
    score = {}

    precision, recall, _ = precision_recall_curve(y_truth, y_predictions)
    score["prauc"] = auc(recall, precision)
    score["precision"] = precision_score(y_truth, y_predictions)
    score["recall"] = recall_score(y_truth, y_predictions)
    score["accuracy"] = accuracy_score(y_truth, y_predictions)

    return score

In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [18]:
df["Predictions"] = le.fit_transform(df["Predictions"])

In [19]:
score_predictions(df["Labels"], df["Predictions"])

/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/evanvance/miniconda3/envs/ids-ai/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'prauc': 0.5, 'precision': 0.0, 'recall': 0.0, 'accuracy': 0.91314247804527}

Note: Precision and Recall are 0.0 because there are no attacks inside the analyzed capture. We can force a computation though with meta-knowledge

I spent quite a few hours trying to put together a way to label the attacks in the netflow generated from the packet capture file, but I was unable to. I think I am missing a key peice in labeling it. I dont have access to the router logs, just captures from an interface that appears to be between the router/firewall and the rest of the network.

Since we know that the benign count is the ammount the model predicted correctly and the mal count is the amount the model predicted incorrectly, we can associate benign count with true positive and mal count with false negatives.

$$
Recall = \frac{TP}{TP + FN}
$$

In [20]:
recall = benign_count / ( benign_count + mal_count)

recall

0.91314247804527

Which is the same as our accuracy since accuracy is

$$
Accuracy = \frac{TP + TN}{ TotalGuesses }
$$

and TN is undefined and $$Total Guesses = benign count + mal count$$

Since 

$$
Precision = \frac{TP}{TP + FP}
$$

and we have no TP, precision is 0 since

$$

0 = \frac{0}{0+malcount}